In [1]:
import json
import sqlite3

In [26]:
    
with open('Resources/survey_responses.json', 'r') as json_file:
    data = json.load(json_file)

In [27]:
print(data)

[{'1': 1, '2': 1, '3': 1, '4': 1, '5': 1, '6': 1, '7': 1, '8': 1, '9': 1, '10': 1, '11': '15', 'timestamp': '2023-09-26T04:34:48.813Z'}]


In [28]:
# Mapping between integer keys and column names
column_mapping = {
    '1': 'A1',
    '2': 'A2',
    '3': 'A3',
    '4': 'A4',
    '5': 'A5',
    '6': 'A6',
    '7': 'A7',
    '8': 'A8',
    '9': 'A9',
    '10': 'A10',
    '11': 'Age_Mons',
    'timestamp': 'Timestamp'
}



In [2]:
conn = sqlite3.connect('Resources/survey_responses.db')

In [3]:
# Create database (if it doesn't already exist)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS survey_responses (
        A1 INTEGER,
        A2 INTEGER,
        A3 INTEGER,
        A4 INTEGER,
        A5 INTEGER,
        A6 INTEGER,
        A7 INTEGER,
        A8 INTEGER,
        A9 INTEGER,
        A10 INTEGER,
        Age_Mons INTEGER,
        Timestamp TEXT,
        Class_ASD_Traits TEXT
    )
''')
conn.commit()


In [4]:
cursor.execute('''
    ALTER TABLE survey_responses
    ADD COLUMN prediction REAL
''')
conn.commit()

In [31]:
# Find new survey responses in the json file and add them to the database
new_responses = [
    {...},  
    {...},  
    # ...
]
# new_responses
for new_response in data:
    # Check if this response already exists in the database
    cursor = conn.cursor()
    cursor.execute('SELECT COUNT(*) FROM survey_responses WHERE Timestamp = ?', (new_response['timestamp'],))
    result = cursor.fetchone()
    response_exists = result[0] > 0  # Check if a response with the same timestamp exists

    if not response_exists:
        # Map integer keys to column names in your database and insert the new response
        mapped_data = {}
        for key, value in new_response.items():
            if key in column_mapping:
                mapped_data[column_mapping[key]] = value

        # Build the INSERT INTO statement dynamically
        columns = ', '.join(mapped_data.keys())
        placeholders = ', '.join(['?'] * len(mapped_data))
        insert_sql = f'INSERT INTO survey_responses ({columns}) VALUES ({placeholders})'

        # Execute the SQL statement
        cursor.execute(insert_sql, tuple(mapped_data.values()))
        conn.commit()

In [5]:
conn.close()